In [3]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

# Create widgets for delay, backoff_factor, max_attempts, and jitter
delay_widget = widgets.IntSlider(value=5, min=0, max=100, description='Delay:')
backoff_factor_widget = widgets.FloatSlider(value=1.5, min=0, max=5, step=0.1, description='Backoff Factor:')
max_attempts_widget = widgets.IntSlider(value=5, min=1, max=20, description='Max Attempts:')
jitter_widget = widgets.FloatSlider(value=1.3, min=1, max=2, step=0.01, description='Jitter:')

def format_time_label(seconds):
    """Convert seconds into 'h m s' format."""
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    if hours:
        return "{}h {}m {}s".format(int(hours), int(minutes), int(seconds))
    elif minutes:
        return "{}m {}s".format(int(minutes), int(seconds))
    else:
        return "{}s".format(int(seconds))

# Create a function to compute jittered wait-time for each attempt and update the plots
def update_plot(delay, backoff_factor, max_attempts, jitter):
    # Define the number of retry attempts
    attempts = np.arange(1, max_attempts + 1)
    
    # Calculate jittered wait-time for each attempt
    random_factors = np.random.uniform(1, jitter, size=max_attempts)
    jittered_wait_times = delay * (backoff_factor ** attempts) + random_factors
    average_wait_times = delay * (backoff_factor ** attempts)
    
    # Calculate cumulative wait-time for each attempt
    total_wait_times = np.cumsum(average_wait_times)
    
    # Compute error (jitter)
    error = jittered_wait_times - average_wait_times
    
    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot the average wait-times and cumulative wait-times on the first subplot
    ax1.errorbar(attempts, average_wait_times, yerr=error, fmt='o-', label='Jittered Wait Time', color='blue')
    ax1.plot(attempts, total_wait_times, marker='x', linestyle='--', color='red', label='Total Wait Time')
    
    # Update y-axis labels to have a custom format
    ax1.set_yticks(np.linspace(0, max(total_wait_times), 6))
    ax1.set_yticklabels([format_time_label(tick) for tick in ax1.get_yticks()])
    
    ax1.set_xlabel('Retry Attempt')
    ax1.set_title('Jittered Wait Time vs. Retry Attempt')
    ax1.legend()
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)
    
    # Update y-axis labels for the bar chart
    bars1 = ax2.bar(attempts, average_wait_times, label='Average Wait Time', color='blue')
    bars2 = ax2.bar(attempts, total_wait_times - average_wait_times, bottom=average_wait_times, label='Total Wait Time Up to Current Attempt', color='red')
    
    ax2.set_yticks(np.linspace(0, max(total_wait_times), 6))
    ax2.set_yticklabels([format_time_label(tick) for tick in ax2.get_yticks()])
    
    # Place labels on top of the bars
    # for bar1, bar2 in zip(bars1, bars2):
    #     height1 = bar1.get_height()
    #     height2 = bar2.get_height()
    #     ax2.text(bar1.get_x() + bar1.get_width()/2., height1,
    #             format_time_label(height1),
    #             ha='center', va='bottom', color='blue')
        
    #     ax2.text(bar2.get_x() + bar2.get_width()/2., height1+height2,
    #             '({})'.format(format_time_label(height1+height2)),
    #             ha='center', va='bottom', color='red')
    
    ax2.set_xlabel('Retry Attempt')
    ax2.set_title('Stacked Bar Chart of Wait Times')
    ax2.legend()
    ax2.grid(True, which='both', linestyle='--', linewidth=0.5)
    
    # Adjust layout
    plt.tight_layout()
    
    # Display the plots
    plt.show()

widgets.interactive(update_plot, delay=delay_widget, backoff_factor=backoff_factor_widget, max_attempts=max_attempts_widget, jitter=jitter_widget)


interactive(children=(IntSlider(value=5, description='Delay:'), FloatSlider(value=1.5, description='Backoff Fa…

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY]) 
# app = dash.Dash(__name__)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label('Delay:'),
            dcc.Slider(id='delay-slider', min=0, max=100, value=30, marks={i: str(i) for i in range(0, 101, 10)}, step=1),
            
            html.Label('Backoff Factor:'),
            dcc.Slider(id='factor-slider', min=0, max=5, value=2, marks={i: str(i) for i in range(6)}, step=0.1),
            
            html.Label('Max Attempts:'),
            dcc.Slider(id='attempts-slider', min=1, max=20, value=10, marks={i: str(i) for i in range(1, 21)}, step=1),
            
            html.Label('Jitter:'),
            dcc.Slider(id='jitter-slider', min=1, max=2, value=1.3, marks={i: f'{i:.2f}' for i in np.linspace(1, 2, 11)}, step=0.01),
        ], width=4),
        
        dbc.Col([
            dcc.Graph(id='wait-time-graph')
        ], width=8)
    ]),
])

def format_time_label(seconds):
    """Convert seconds into 'h m s' format."""
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    if hours:
        return "{}h {}m {}s".format(int(hours), int(minutes), int(seconds))
    elif minutes:
        return "{}m {}s".format(int(minutes), int(seconds))
    else:
        return "{}s".format(int(seconds))

@app.callback(
    Output('wait-time-graph', 'figure'),
    [Input('delay-slider', 'value'),
     Input('factor-slider', 'value'),
     Input('attempts-slider', 'value'),
     Input('jitter-slider', 'value')]
)
def update_figure(delay, backoff_factor, max_attempts, jitter):
    # Define the number of retry attempts
    attempts = np.arange(1, max_attempts + 1)
    
    # Calculate jittered wait-time for each attempt
    random_factors = np.random.uniform(1, jitter, size=max_attempts)
    jittered_wait_times = delay * (backoff_factor ** attempts) * random_factors
    average_wait_times = delay * (backoff_factor ** attempts)
    
    # Calculate cumulative wait-time for each attempt
    total_wait_times = np.cumsum(average_wait_times)
    
    # Create the figure with subplots
    fig = go.Figure()

    # Error bars
    error = jittered_wait_times - average_wait_times

    # Add jittered wait-time line to the figure
    fig.add_trace(go.Scatter(
        x=attempts,
        y=jittered_wait_times,
        mode='lines+markers',
        name='Jittered Wait Time',
        line=dict(color='blue'),
        error_y=dict(
            type='data',
            array=error,
            visible=True
        )
    ))

    # Add cumulative wait-time line to the figure
    fig.add_trace(go.Scatter(
        x=attempts,
        y=total_wait_times,
        mode='lines+markers',
        name='Total Wait Time',
        line=dict(color='red', dash='dash')
    ))

    # Set y-axis labels
    fig.update_layout(
        plot_bgcolor='#2C3E50',
        paper_bgcolor='#2C3E50',
        font=dict(color='white'),
        xaxis_title='Retry Attempt',
        yaxis_title='Time',
        yaxis_tickvals=np.linspace(0, max(total_wait_times), 6),
        yaxis_ticktext=[format_time_label(tick) for tick in np.linspace(0, max(total_wait_times), 6)]
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/65/57wn7ttx42713rnb3qc3k2ph0000gn/T/ipykernel_25538/939070281.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/65/57wn7ttx42713rnb3qc3k2ph0000gn/T/ipykernel_25538/939070281.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
